In [1]:
import pandas as pd 
import numpy as np 
import os
import pickle
import re 
import sys
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,plot_confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit,LeaveOneOut,KFold
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score

In [2]:
sys.path.append('.')
sys.path.append('..')

In [3]:
from Fourier import fourier_feature_fusion as eff
from Wavelet import wavelet_feature_fusion as wff 

In [4]:
user_no=2
user_folder='../../../User2/'
last_index_even=41 # For even user numbers
last_index_odd=40  # For odd user numbers
increment=2        # Number for the increment of the user numbers

In [5]:
def getLabel(label):
    new_label=[]
    for i in range(len(label)):
        if(label[i]>4.5):
            new_label.append(1)
        else:
            new_label.append(0)
    return new_label

In [6]:
# clean data
def clean_data(dataset, target):
    del_rows = []
    for i in range(len(dataset)):
        if(np.isnan(dataset[i]).sum() > 0):
            del_rows.append(i)
            
    dataset = np.delete(dataset, del_rows, axis=0)
    target = np.delete(target, del_rows, axis=0)
    
    return dataset,target

In [7]:
window_size = 512
step_size = 512

In [8]:
arousal=[]
valence=[]
dominance=[]
liking=[]

In [9]:
def get_full_features(user_no, window_size,step_size):
    
    ar_dict = {}
    val_dict = {}
    dom_dict = {}
    lik_dict = {}
    
    user_folder='../../../User'+ str(user_no) + '/'
    
    user_csv= os.listdir(user_folder)
    
    user_csv = user_csv[1:]
    
    user_labels_folder = user_folder + 'Label/'
    
    user_labels=os.listdir(user_labels_folder)
    
    all_data=[]
    all_labels=[]
    
    for csv in user_csv:
        all_data.append(csv)
    
    for lab in user_labels:
        all_labels.append(lab)
        
    final_features=[]
    final_labels=[]
        
    for i in range(0,len(all_data)):
        
        data = pd.read_csv(user_folder + all_data[i])
        cols = data.columns
        cols = cols[:14]
        labels=pd.read_csv(user_labels_folder + all_labels[i])
        labels=labels.values[0][:4]
        
        start = 0;
        while start + window_size < data.shape[0]:
            
            temp_array = []
            temp_data = [] 
            
            for i in cols:
                X = data[i][start : start + window_size]
                features_fourier = eff.power_spectrum(X)
                features_wavelet = wff.wavelet_energy(X)
                features_fourier = np.array(features_fourier).ravel()
                features_wavelet = np.array(features_wavelet).ravel()
                temp_array.append(np.concatenate((features_fourier,features_wavelet),axis=0).ravel())
                  
            temp_array=np.array(temp_array).ravel()
            final_features.append(temp_array) 
            final_labels.append(labels)
            start = start + step_size
                
    final_features = np.array(final_features)
    final_labels = np.array(final_labels)
#     print(final_features.shape)
#     print(final_labels.shape)
    
    arousal_dataset=final_features
    valence_dataset=final_features
    dominance_dataset=final_features
    liking_dataset=final_features
    
    arousal_labels=final_labels[:,0]
    valence_labels=final_labels[:,1]
    dominance_labels=final_labels[:,2]
    liking_labels=final_labels[:,3]
    
    
    arousal_dataset,arousal_labels = clean_data(arousal_dataset,arousal_labels)
    valence_dataset,valence_labels = clean_data(valence_dataset,valence_labels)
    dominance_dataset ,dominance_labels = clean_data(dominance_dataset,dominance_labels)
    liking_dataset,liking_labels = clean_data(liking_dataset,liking_labels)
    
    
    arousal_labels = getLabel(arousal_labels)
    valence_labels = getLabel(valence_labels)
    dominance_labels = getLabel(dominance_labels)
    liking_labels = getLabel(liking_labels)
    
    ar_count=len(set(arousal_labels))
    if ar_count==1:
        ar_dict[user_no]=['Accuracy: '+str(100),'F1-Score: '+str(1)]
        arousal.append(ar_dict)
    
    else:
        sm1 = SMOTE()
        arousal_dataset,arousal_labels = sm1.fit_resample(arousal_dataset,arousal_labels)
        x_a_train,x_a_test,y_a_train,y_a_test = train_test_split(arousal_dataset,arousal_labels,test_size=0.3,random_state=42)
        ar_model = SVC(kernel="rbf",C = 10)
        ar_model.fit(x_a_train,y_a_train)
        pred_a = ar_model.predict(x_a_test)
        ar_acc=accuracy_score(pred_a,y_a_test)
        ar_f1=f1_score(y_a_test, pred_a, average='macro')
        
        ar_dict[user_no]=['Accuracy: '+str(ar_acc),'F1-Score: '+str(ar_f1)]
        arousal.append(ar_dict)
        
        
    val_count=len(set(valence_labels))
    if val_count==1:
        val_dict[user_no]=['Accuracy: '+str(100),'F1-Score: '+str(1)]
        valence.append(val_dict)
        
    else:
        sm2 = SMOTE()
        valence_dataset,valence_labels = sm2.fit_resample(valence_dataset,valence_labels)
        x_v_train,x_v_test,y_v_train,y_v_test = train_test_split(valence_dataset,valence_labels,test_size=0.3,random_state=42)
        val_model = SVC(kernel="rbf",C = 10)
        val_model.fit(x_v_train,y_v_train)
        pred_v = val_model.predict(x_v_test)
        val_acc=accuracy_score(pred_v,y_v_test)
        val_f1=f1_score(y_v_test, pred_v, average='macro')
        
        val_dict[user_no]=['Accuracy: '+str(val_acc),'F1-Score: '+str(val_f1)]
        valence.append(val_dict)
        
        
    dom_count=len(set(dominance_labels))
    if dom_count==1:
        dom_dict[user_no]=['Accuracy: '+str(100),'F1-Score: '+str(1)]
        dominance.append(dom_dict)
    
    else:
        sm3 = SMOTE()
        dominance_dataset,dominance_labels = sm3.fit_resample(dominance_dataset,dominance_labels)
        x_d_train,x_d_test,y_d_train,y_d_test = train_test_split(dominance_dataset,dominance_labels,test_size = 0.3,random_state = 42)
        dom_model = SVC(kernel="rbf",C = 10)
        dom_model.fit(x_d_train,y_d_train)
        pred_d = dom_model.predict(x_d_test)
        dom_acc=accuracy_score(pred_d,y_d_test)
        dom_f1=f1_score(y_d_test, pred_d, average='macro')
        
        dom_dict[user_no]=['Accuracy: '+str(dom_acc),'F1-Score: '+str(dom_f1)]
        dominance.append(dom_dict)
        
    
    lik_count=len(set(liking_labels))
    if lik_count==1:
        lik_dict[user_no]=['Accuracy: '+str(100),'F1-Score: '+str(1)]
        liking.append(lik_dict)

    else:
        sm4 = SMOTE()
        liking_dataset,liking_labels = sm4.fit_resample(liking_dataset,liking_labels)
        x_l_train,x_l_test,y_l_train,y_l_test = train_test_split(liking_dataset,liking_labels,test_size = 0.3,random_state = 42)
        lik_model = SVC(kernel="rbf",C = 10)
        lik_model.fit(x_l_train,y_l_train)
        pred_l = lik_model.predict(x_l_test)
        lik_acc=accuracy_score(pred_l,y_l_test)
        lik_f1=f1_score(y_l_test, pred_l, average='macro')
 
        lik_dict[user_no]=['Accuracy: '+str(lik_acc),'F1-Score: '+str(lik_f1)]
        liking.append(lik_dict)
    
    
    print('User:' + str(user_no)+ " "+"Done")

In [10]:
for i in range(user_no,last_index_even,2):
    get_full_features(i,window_size,step_size)

User:2 Done
User:4 Done
User:6 Done
User:8 Done
User:10 Done
User:12 Done
User:14 Done
User:16 Done
User:18 Done
User:20 Done
User:22 Done
User:24 Done
User:26 Done
User:28 Done
User:30 Done
User:32 Done
User:34 Done
User:36 Done
User:38 Done
User:40 Done


In [11]:
def get_dict(res,idx):
    user_no = []
    accuracy = []
    f1_score = []

    for i in range(len(res)):
        user_no.append(idx)
        acc = res[i][idx][0]
        acc = acc.split(' ')
        acc = float(acc[1])
        accuracy.append(acc)
    
        f1 = res[i][idx][1]
        f1 = f1.split(' ')
        f1 = float(f1[1])
        f1_score.append(f1)
        idx += 2
    f_dict = {'user_no':user_no, 'accuracy':accuracy,'f1_score':f1_score}
    return f_dict

In [13]:
f_dict = get_dict(arousal,2)    
df = pd.DataFrame(data=f_dict)
df.to_csv('./even_arousal.csv',index=False)
df

,user_no,accuracy,f1_score
0,2,0.882510,0.882334
1,4,0.965632,0.965632
2,6,0.978771,0.978767
3,8,0.930851,0.930281
4,10,0.950376,0.950275
5,12,0.964953,0.964950
6,14,0.918639,0.918639
7,16,0.934492,0.934486
8,18,0.972936,0.972934
9,20,0.906841,0.906718


In [14]:
f_dict = get_dict(valence,2)
df = pd.DataFrame(data=f_dict)
df.to_csv('./even_valence.csv',index=False)
df

,user_no,accuracy,f1_score
0,2,0.898413,0.898396
1,4,0.861224,0.860630
2,6,0.932710,0.932563
3,8,0.709091,0.708223
4,10,0.873563,0.873517
5,12,0.891447,0.890647
6,14,0.795132,0.795048
7,16,0.883142,0.883090
8,18,0.949458,0.949416
9,20,0.847423,0.846529


In [15]:
f_dict = get_dict(dominance,2)
df = pd.DataFrame(data=f_dict)
df.to_csv('./even_dominance.csv',index=False)
df

,user_no,accuracy,f1_score
0,2,0.915423,0.915330
1,4,0.834320,0.833776
2,6,0.968362,0.968340
3,8,0.684211,0.684113
4,10,0.984899,0.984892
5,12,0.939271,0.939151
6,14,0.984668,0.984659
7,16,0.932524,0.932503
8,18,0.983312,0.983276
9,20,0.810865,0.810459


In [16]:
f_dict = get_dict(liking,2)
df = pd.DataFrame(data=f_dict)
df.to_csv('./even_liking.csv',index=False)
df

,user_no,accuracy,f1_score
0,2,0.956372,0.956330
1,4,0.961722,0.961722
2,6,0.973225,0.973221
3,8,0.826087,0.823077
4,10,0.988739,0.988739
5,12,0.929134,0.928975
6,14,0.920588,0.920157
7,16,0.956318,0.956271
8,18,0.960219,0.960205
9,20,0.897710,0.897364
